# Table of Contents
 <p><div class="lev1 toc-item"><a href="#爬取单页内容" data-toc-modified-id="爬取单页内容-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>爬取单页内容</a></div><div class="lev1 toc-item"><a href="#读取单页内容" data-toc-modified-id="读取单页内容-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>读取单页内容</a></div><div class="lev1 toc-item"><a href="#爬取多页内容" data-toc-modified-id="爬取多页内容-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>爬取多页内容</a></div><div class="lev2 toc-item"><a href="#首页-&gt;-财经频道-&gt;-要闻-&gt;-公司新闻" data-toc-modified-id="首页->-财经频道->-要闻->-公司新闻-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>首页 &gt; 财经频道 &gt; 要闻 &gt; 公司新闻</a></div><div class="lev2 toc-item"><a href="#首页-&gt;-财经频道-&gt;-要闻-&gt;-上市公司-&gt;-列表" data-toc-modified-id="首页->-财经频道->-要闻->-上市公司->-列表-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>首页 &gt; 财经频道 &gt; 要闻 &gt; 上市公司 &gt; 列表</a></div><div class="lev1 toc-item"><a href="#读取多页内容" data-toc-modified-id="读取多页内容-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>读取多页内容</a></div><div class="lev2 toc-item"><a href="#读取-3.1" data-toc-modified-id="读取-3.1-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>读取 3.1</a></div><div class="lev2 toc-item"><a href="#读取-3.2" data-toc-modified-id="读取-3.2-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>读取 3.2</a></div><div class="lev2 toc-item"><a href="#集合数据" data-toc-modified-id="集合数据-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>集合数据</a></div><div class="lev1 toc-item"><a href="#标注数据" data-toc-modified-id="标注数据-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>标注数据</a></div><div class="lev2 toc-item"><a href="#标注细节" data-toc-modified-id="标注细节-51"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>标注细节</a></div><div class="lev3 toc-item"><a href="#实体标注细节" data-toc-modified-id="实体标注细节-511"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>实体标注细节</a></div><div class="lev3 toc-item"><a href="#关系标注细节" data-toc-modified-id="关系标注细节-512"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>关系标注细节</a></div>

# 爬取单页内容

In [101]:
import requests
from requests.exceptions import RequestException
import re
import json

In [102]:
def get_one_page(url):
    try:
        response = requests.get(url).content.decode('utf-8')
        if requests.get(url).status_code == 200:
            return response
        return None
    except RequestException:
        return None

def write_to_file(content):
    with open('single_page.json', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')
        f.close()
        
def parse_one_page(html):
    pattern = re.compile('<p class="title">.*?<a href=".*?">(.*?)</a>.*?<p class="info" title="(.*?)">.*?<p class="time">(.*?)</p>'
                         , re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield {
            'title': item[0],
            'abstract': item[1],
            'time': item[2][:-6]
        }    

In [103]:
url = 'http://finance.eastmoney.com/news/cgsxw_23.html'
html = get_one_page(url)
single_page = parse_one_page(html)
for s in single_page:
    write_to_file(s)

# 读取单页内容

In [4]:
import json
import pickle

In [14]:
with open('single_page.json') as f:
    lines = f.readlines()
    
title = []
abstract = []
time = []
for line in lines:
    item = json.loads(line)
    title.append(item['title'].strip())
    abstract.append(item['abstract'].strip())
    time.append('2017年' + item['time'].strip()[:-6])
    
tokens = (title, abstract, time)

In [15]:
len(title), len(abstract), set(time)

(18, 18, {'2017年06月12日'})

In [18]:
with open('tokens.pkl', 'wb') as fp:
    pickle.dump(tokens, fp)

In [19]:
with open('tokens.pkl', 'rb') as fp:
    title, abstract, time = pickle.load(fp)

In [21]:
title[0]

'日播时尚业绩滑坡库存高 女装三年13起抽检不合格'

In [22]:
abstract[0]

'中国经济网记者就上述问题向公司证券事务部发去采访函，截至发稿时未收到回复。中孚信息是专注于信息安全领域的高新技术企业，主营业务为信息安全产品研发、生产、销售及信息安全服务。公司主要产品及服务包括信息安全保密产品、商用密码产品、信息安全服务。信息安全保密产品服务对象主要为党政机关、军工企业、中央企业、科研院所等，商用密码产品服务领域主要为金融、税务、工商等。信息安全服务为涉密信息系统集成服务。公司坚持自主创新，研发了主要面向党政机关、军工企业、中央企业、科研院所的信息安全保密产品，部分产品处于国内领先地位。'

In [23]:
time[0]

'2017年06月12日'

# 爬取多页内容

## 首页 > 财经频道 > 要闻 > 公司新闻

In [1]:
import requests
from multiprocessing import Pool
from requests.exceptions import RequestException
import re
import json
import time

In [2]:
def get_one_page(url):
    try:
        response = requests.get(url).content.decode('utf-8')
        if requests.get(url).status_code == 200:
            return response
        return None
    except RequestException:
        return None

def write_to_file(content):
    with open('all_pages.json', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')
        f.close()
        
def parse_one_page(html):
    pattern = re.compile('<p class="title">.*?<a href=".*?">(.*?)</a>.*?<p class="info" title="(.*?)">.*?<p class="time">(.*?)</p>'
                         , re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield {
            'title': item[0],
            'abstract': item[1],
            'time': item[2]
        }    

def all_pages(num):
    url = 'http://finance.eastmoney.com/news/cgsxw_' + str(num) + '.html'
    html = get_one_page(url)
    for item in parse_one_page(html):
        write_to_file(item)

In [3]:
pool = Pool()
pool.map(all_pages, [i+1 for i in range(25)])

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [4]:
#start = time.time()
#for i in range(23):
#    all_pages(i+1)
#end = time.time()
#print('Time-consuming: %.3fs' % (end - start))

## 首页 > 财经频道 > 要闻 > 上市公司 > 列表

In [5]:
import requests
from multiprocessing import Pool
from requests.exceptions import RequestException
import re
import json
import time

In [6]:
def get_one_page(url):
    try:
        response = requests.get(url).content.decode('utf-8')
        if requests.get(url).status_code == 200:
            return response
        return None
    except RequestException:
        return None

def write_to_file(content):
    with open('all_pages_2.json', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False, sort_keys=True) + '\n')
        f.close()
        
def parse_one_page(html):
    pattern = re.compile('<p class="title">.*?<a href=".*?">(.*?)</a>.*?<p class="info" title="(.*?)">.*?<p class="time">(.*?)</p>'
                         , re.S)
    items = re.findall(pattern, html)
    for item in items:
        yield {
            'title': item[0],
            'abstract': item[1],
            'time': item[2]
        }    

def all_pages(num):
    url = 'http://finance.eastmoney.com/news/cssgs_' + str(num) + '.html'
    html = get_one_page(url)
    for item in parse_one_page(html):
        write_to_file(item)

In [7]:
pool = Pool()
pool.map(all_pages, [i+1 for i in range(25)])

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
#start = time.time()
#for i in range(9):
#    all_pages(i+1)
#end = time.time()
#print('Time-consuming: %.3fs' % (end - start))

# 读取多页内容

## 读取 3.1

In [8]:
import json
import pickle

In [9]:
with open('all_pages.json') as f:
    lines = f.readlines()
       
title = []
abstract = []
time = []
for line in list(set(lines)):
    item = json.loads(line)
    title.append(item['title'].strip())
    abstract.append(item['abstract'].strip())
    time.append('2017年' + item['time'].strip()[:-6])
    
tokens = (title, abstract, time)

In [10]:
#with open('all_tokens.pkl', 'wb') as fp:
#    pickle.dump(tokens, fp)

In [11]:
#with open('all_tokens.pkl', 'rb') as fp:
#    title, abstract, time = pickle.load(fp)

In [12]:
len(lines), len(title), len(set(abstract)), len(time)

(5532, 3724, 3692, 3724)

In [13]:
len(set(lines)), len(set(title)), len(set(abstract)), len(set(time))

(3724, 3714, 3692, 17)

In [14]:
set(time)

{'2017年08月07日',
 '2017年08月08日',
 '2017年08月09日',
 '2017年08月10日',
 '2017年08月11日',
 '2017年08月12日',
 '2017年08月13日',
 '2017年08月14日',
 '2017年08月15日',
 '2017年08月16日',
 '2017年08月17日',
 '2017年08月18日',
 '2017年08月19日',
 '2017年08月20日',
 '2017年08月30日',
 '2017年08月31日',
 '2017年09月01日'}

In [15]:
title[0]

'深圳能源：子公司将出租、出售住宅项目 预增年度净利4亿元'

In [16]:
abstract[0]

'【深圳能源：子公司将出租、出售住宅项目 预增年度净利4亿元】深圳能源(000027) 16日晚公告，控股子公司妈湾公司拟将开发建设的深圳市南山区电力花园二期住宅项目整体定向、分期向深圳市人才安居集团有限公司租售。（证券时报网）'

In [17]:
time[0]

'2017年08月16日'

## 读取 3.2

In [18]:
import json
import pickle

In [19]:
with open('all_pages_2.json') as f:
    lines = f.readlines()
       
title = []
abstract = []
time = []
for line in list(set(lines)):
    item = json.loads(line)
    title.append(item['title'].strip())
    abstract.append(item['abstract'].strip())
    time.append('2017年' + item['time'].strip()[:-6])
    
tokens = (title, abstract, time)

In [20]:
#with open('all_tokens_2.pkl', 'wb') as fp:
#    pickle.dump(tokens, fp)

In [21]:
#with open('all_tokens_2.pkl', 'rb') as fp:
#    title, abstract, time = pickle.load(fp)

In [22]:
len(lines), len(title), len(abstract), len(time)

(3497, 1623, 1623, 1623)

In [23]:
len(set(lines)), len(set(title)), len(set(abstract)), len(set(time))

(1623, 1620, 1588, 17)

In [24]:
set(time)

{'2017年08月07日',
 '2017年08月08日',
 '2017年08月09日',
 '2017年08月10日',
 '2017年08月11日',
 '2017年08月12日',
 '2017年08月13日',
 '2017年08月14日',
 '2017年08月15日',
 '2017年08月16日',
 '2017年08月17日',
 '2017年08月18日',
 '2017年08月19日',
 '2017年08月29日',
 '2017年08月30日',
 '2017年08月31日',
 '2017年09月01日'}

In [25]:
title[0]

'出资8亿控股能投风电 *ST川化产业转型再进一步'

In [26]:
abstract[0]

'【出资8亿控股能投风电 *ST川化产业转型再进一步】在四川省大力推进上市公司并购重组、全力提高国资证券化率的双重背景下，*ST川化筹划许久的资产收购方案于今日出炉，公司拟以现金方式收购能投风电55%股权，由此快速进入风电运营领域，也为公司未来的进一步转型发展奠定基础。（上海证券报）'

In [27]:
time[0]

'2017年08月11日'

## 集合数据

In [28]:
import json
import pickle

In [29]:
title = []
abstract = []

In [30]:
with open('all_pages.json') as f1:
    lines1 = f1.readlines()
       
with open('all_pages_2.json') as f2:
    lines2 = f2.readlines()
        
for line in list(set(lines1)):
    item = json.loads(line)
    title.append(item['title'].strip())
    abstract.append(item['abstract'].strip())

for line in list(set(lines2)):
    item = json.loads(line)
    title.append(item['title'].strip())
    abstract.append(item['abstract'].strip())
    
title = list(set(title))
abstract = list(set(abstract))

In [31]:
with open('title.pkl', 'wb') as fp:
    pickle.dump(title, fp)
with open('abstract.pkl', 'wb') as fp:
    pickle.dump(abstract, fp)

In [32]:
#with open('title.pkl', 'rb') as fp:
#    title = pickle.load(fp)
#with open('abstract.pkl', 'rb') as fp:
#    abstract = pickle.load(fp)

In [33]:
len(title), len(abstract)

(4190, 4032)

In [34]:
title[1]

'凤形股份股东计划减持不超176万股'

In [35]:
abstract[1]

'【双环传动：上半年净利增约四成 推2000万股限制性股票】双环传动(002472)8月20日晚披露半年报，公司上半年实现营业总收入11.21亿元，同比增长43%；净利润1.14亿元，同比增38.46%。此外，公司拟向激励对象授予限制性股票2000万股，占总股本的2.96%。股票来源为二级市场上回购股份和向激励对象定向发行股份，目前公司已回购1000万股，将全部作为实施公司激励计划的部分股票来源。（证券时报网）'

# 标注数据

In [36]:
import json
import pickle

```python
with open('abstract_data/01.pkl', 'rb') as fp:
    abstract = pickle.load(fp)
with open('abstract_data/02.pkl', 'rb') as fp:
    abstract_2 = pickle.load(fp)
abstract.extend(abstract_2)
abstract = list(set(abstract))
len(abstract)
```

In [58]:
with open('title_data/11.pkl', 'rb') as fp:
    title = pickle.load(fp)
title = list(set(title))
len(title)

4190

In [59]:
title

['百德国际完成收购佳力科技(i-制造)',
 '凤形股份股东计划减持不超176万股',
 '双环科技实控人或变更',
 '万洲国际上半年归于双汇实业集团股东的净利减少11.71%至11.25亿元',
 '金融街前三季预增45%-80%',
 '中国电信将在雄安新区建设国家骨干网核心节点 5G创新示范网启动建设',
 '联想第一财季净亏损7200万美元',
 '九有股份：大股东筹划重大事项 或致公司控制权变更',
 '光大证券：上半年净利同比降18% 证金公司持股达4.99%',
 '万德金融完成收购衡资产管理51%股份溢价11.11%发行1亿股代价股份',
 '朗姿股份拟12亿元参与设立时尚产业合伙企业',
 '嘉寓股份：中标1.92亿光伏扶贫电站项目',
 '保千里：实控人及其一致行动人遭证监会行政处罚',
 '卖卖卖后又搞大动作 这家万达系公司一天大涨20%',
 '基金重仓58只“创蓝筹”股票 中邮基金任泽松与国家队“齐恋”昆仑万维',
 '融资111亿 北汽新能源蓄势待发',
 '沪股通连续三天净卖出贵州茅台 力度仍温和',
 '百灵达国际控股建议改名“新源万恒控股”',
 '农发行禹城违法向客户收取投融资财务顾问费',
 '先达国际物流：圆通预期3星期内就外汇审批向外汇管理部门提交申请',
 '500元！茅台成25年A股第一高价股 总市值超半个贵州GDP',
 '晶科电子上半年营收超4亿 净利同增逾六成',
 '中国支付通完成发行第二批认购股份已发行股本增至16.61亿股',
 '投融长富延长订立正式协议涉收购中油星光不少于51%的股本',
 '盛运环保：子公司获阿勒泰地区生活垃圾焚烧发电项目',
 '新力金融控股股东发布新增持计划',
 '天源迪科：上半年净利同比增长303%',
 '龙头股份、申达股份：纺织集团与东方国际（集团）实施联合重组',
 '泰达股份控股股东拟变更为泰达控股',
 '科地农业一季度扭亏为盈股东应占溢利159万港元',
 '伊利中报：上半年净利润33.68亿元',
 '交通银行聊城分行业务审查违规',
 '洲际油气控股股东股份被全部冻结',
 '坚瑞沃能4名股东拟合计减持不超4315.57万股',
 '班尼路长裤检出禁用染料 与皮肤接触可诱发恶性疾病',
 '萤石价格有所上升 金石资源半年报增两成',
 '贵人鸟转型被指缺乏产业

In [37]:
with open('abstract_data/11.pkl', 'rb') as fp:
    abstract = pickle.load(fp)
abstract = list(set(abstract))
len(abstract)

4032

In [38]:
def write_to_file(content):
    with open('train/data.json', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')
        f.close()
        
def gen_data(data):
    for item in data:
        yield {
            'abstract': item,
            'eneity': ["", ""],
            'relation': ""
        }

In [54]:
def write_to_file(content):
    with open('train/data.json', 'a', encoding='utf-8') as f:
        f.write(json.dumps(content, ensure_ascii=False) + '\n')
        f.close()
        
def gen_data(data):
    for item in data:
        yield {
            'abstract': item
        }

In [55]:
for i in gen_data(abstract):
    write_to_file(i)

## 标注细节

### 实体标注细节

### 关系标注细节

- 股权转让
- 无相关关系
- #收到监管措施事先告知书（证监会）
- 收购
- 合作
- 损失
- 失去控制权
- 提供技术支持
- 认购股票
- 购入股权
- #存在多种关系：子公司，
- 

In [47]:
with open('train/trian_data.json') as f:
    lines = f.readlines()
       
abstract = []
eneity = []

for line in lines:
    item = json.loads(line)
    abstract.append(item['abstract'])
    eneity.append(item['eneity'])

In [53]:
len(abstract)

21273

In [48]:
len(set(abstract))

21273

In [50]:
abstract[:3]

['【东富龙：向迪安诊断转让重庆产研院股权】东富龙(300171)6月1日晚公告，公司下属全资子公司东富龙医疗与迪安诊断(300244)签署股权转让协议，约定东富龙医疗以5035万元，向迪安诊断转让重庆精准医疗产业技术研究院有限公司(简称重庆产研院)100%股权。（证券时报网）',
 '【康缘药业：筹划股权激励事项 明起停牌】康缘药业(600557)6月5日晚间公告，公司正在筹划股权激励事项，公司股票将自6月6日起停牌。公司将尽快确定是否进行上述重大事项，并于5个工作日内公告并复牌。（证券时报）',
 '昨日晚间，国海证券发布公告称，公司今年5月份营业收入1.21亿元，净利润1665.67万元；分别环比下滑26%、60%；分别同比下滑45%、81%。国海证券今年5月份收到证监会行政监管措施事先告知书，拟自行政监督管理措施正式下发之日起一年内，暂停国海证券新开证券账户等三项业务。']

In [52]:
abstract = list(set(abstract))

In [50]:
abstract[0]

'【东富龙：向迪安诊断转让重庆产研院股权】东富龙(300171)6月1日晚公告，公司下属全资子公司东富龙医疗与迪安诊断(300244)签署股权转让协议，约定东富龙医疗以5035万元，向迪安诊断转让重庆精准医疗产业技术研究院有限公司(简称重庆产研院)100%股权。（证券时报网）'

In [44]:
len(eneity)

21293

In [52]:
eneity[0]

['东富龙', '迪安诊断', '东富龙医疗', '重庆产研院']